In [1]:
!git clone https://github.com/Madhuri301991/Deep_Learning_Assignment

Cloning into 'Deep_Learning_Assignment'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 92 (delta 20), reused 44 (delta 12), pack-reused 33
Receiving objects: 100% (92/92), 122.49 MiB | 17.86 MiB/s, done.
Resolving deltas: 100% (25/25), done.


In [2]:
!pip install tensorflow

In [3]:
!pip install gensim scikit-learn pandas numpy

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from keras.regularizers import l2
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, SpatialDropout1D, Conv1D, GlobalMaxPooling1D, SimpleRNN, GlobalAveragePooling1D, Flatten, Concatenate
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix,f1_score, precision_score, recall_score
from gensim.models import KeyedVectors
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle


In [5]:
def preprocess(train_df, test_df):
    train_df['Lyrics'] = train_df['Lyrics'].fillna("")
    test_df['Lyrics'] = test_df['Lyrics'].fillna("")


    train_df.dropna(subset=["Lyrics", "Genre"], inplace=True)
    test_df.dropna(subset=["Lyrics","Genre"], inplace=True)


    train_df['Lyrics'] = train_df['Lyrics'].astype(str).str.lower()
    test_df['Lyrics'] = test_df['Lyrics'].astype(str).str.lower()

    return train_df, test_df

In [6]:
def vectorization(train_df,test_df):
  tfidf_vectorizer = TfidfVectorizer(max_features=5000)
  train_vec = tfidf_vectorizer.fit_transform(train_df['Lyrics'])
  test_vec = tfidf_vectorizer.transform(test_df['Lyrics'])
  return train_vec,test_vec

In [7]:
def enc(train_df,test_df):
  label_encoder=LabelEncoder()
  train_encoded_labels=label_encoder.fit_transform(train_df["Genre"])
  test_encoded_labels=label_encoder.transform(test_df["Genre"])
  return train_encoded_labels,test_encoded_labels

In [8]:
def encoding(train_df,test_df):
  label_encoder = LabelEncoder()
  train_encoded_labels = label_encoder.fit_transform(train_df["Genre"])
  train_labels = to_categorical(train_encoded_labels)

  test_encoded_labels=label_encoder.transform(test_df['Genre'])
  test_labels = to_categorical(test_encoded_labels)
  return train_labels,test_labels,label_encoder

In [9]:
def tokenizing(train_df,test_df):

  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(train_df['Lyrics'])
  max_length = max(max(len(s.split()) for s in train_df['Lyrics']), max(len(s.split()) for s in test_df['Lyrics']))
  vocab_size = len(tokenizer.word_index) + 1

  train_sequences = tokenizer.texts_to_sequences(train_df['Lyrics'])
  train_data = pad_sequences(train_sequences, maxlen=max_length)
  test_sequences = tokenizer.texts_to_sequences(test_df['Lyrics'])
  test_data = pad_sequences(test_sequences, maxlen=max_length)
  return train_data,test_data,max_length,vocab_size

In [10]:
import os
path=os.getcwd()

In [11]:
#setting path
import sys
sys.path.append(path)

**CNN model**

In [12]:
train_df = pd.read_csv('Deep_Learning_Assignment/Part1train.csv')
test_df = pd.read_csv('Deep_Learning_Assignment/test_half.csv')

train_df,test_df=preprocess(train_df,test_df)
train_labels,test_labels,label_encoder=encoding(train_df,test_df)
train_data,test_data,max_length,vocab_size=tokenizing(train_df,test_df)

loaded_model = load_model("Deep_Learning_Assignment/CNN_model.h5")


y_pred_prob=loaded_model.predict(test_data)
y_pred=np.argmax(y_pred_prob,axis=1)
y_true=np.argmax(test_labels,axis=1)
f1=f1_score(y_true,y_pred,average='weighted')
print(f"F1 score:{f1}")


precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')

print(f'Precision: {precision}')
print(f'Recall: {recall}')

test_loss, test_accuracy = loaded_model.evaluate(test_data, test_labels)
print(f'Test accuracy: {test_accuracy}')
print(f'Loss:{test_loss}')

1/1 [==============================] - 2s 2s/step
F1 score:0.07888212756366914
Precision: 0.07277205553067621
Recall: 0.13793103448275862


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1/1 [==============================] - 0s 472ms/step - loss: 2.8100 - accuracy: 0.1379
Test accuracy: 0.13793103396892548
Loss:2.810016393661499


**RNN Model**

In [13]:
train_df = pd.read_csv('Deep_Learning_Assignment/Part1train.csv')
test_df = pd.read_csv('Deep_Learning_Assignment/test_half.csv')

train_df,test_df=preprocess(train_df,test_df)
train_labels,test_labels,label_encoder=encoding(train_df,test_df)
train_data,test_data,max_length,vocab_size=tokenizing(train_df,test_df)

loaded_model = load_model("Deep_Learning_Assignment/RNN_new_model.h5")


y_pred_prob=loaded_model.predict(test_data)
y_pred=np.argmax(y_pred_prob,axis=1)
y_true=np.argmax(test_labels,axis=1)
f1=f1_score(y_true,y_pred,average='weighted')
print(f"F1 score:{f1}")

precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')

print(f'Precision: {precision}')
print(f'Recall: {recall}')
test_loss, test_accuracy = loaded_model.evaluate(test_data, test_labels)
print(f'Test accuracy: {test_accuracy}')
print(f'Loss:{test_loss}')

1/1 [==============================] - 0s 481ms/step
F1 score:0.22246941045606228
Precision: 0.16420361247947454
Recall: 0.3448275862068966


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1/1 [==============================] - 1s 616ms/step - loss: 2.3087 - accuracy: 0.3448
Test accuracy: 0.3448275923728943
Loss:2.3086776733398438


**Basic Modeling with 5 layer + Skip Connection**

In [14]:
!pwd

/content


In [15]:
from tensorflow.keras.datasets import cifar100
import numpy as np
import os


(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')


np.random.seed(42)
class_indices = np.arange(100)
np.random.shuffle(class_indices)

block_1_classes = class_indices[:50]
block_2_classes = class_indices[50:]

def filter_by_classes(x, y, classes):
    mask = np.isin(y.flatten(), classes)
    return x[mask], y[mask]

x_train_block_1, y_train_block_1 = filter_by_classes(x_train, y_train, block_1_classes)
x_test_block_1, y_test_block_1 = filter_by_classes(x_test, y_test, block_1_classes)

x_train_block_2, y_train_block_2 = filter_by_classes(x_train, y_train, block_2_classes)
x_test_block_2, y_test_block_2 = filter_by_classes(x_test, y_test, block_2_classes)

# Save to disk
data_path = "Deep_Learning_Assignment/dataset/PartB"
if not os.path.exists(data_path):
    os.makedirs(data_path)

np.savez(data_path + "block_1", x_train=x_train_block_1, y_train=y_train_block_1, x_test=x_test_block_1, y_test=y_test_block_1)
np.savez(data_path + "block_2", x_train=x_train_block_2, y_train=y_train_block_2, x_test=x_test_block_2, y_test=y_test_block_2)

print("Blocks saved to disk.")

169001437/169001437 [==============================] - 3s 0us/step
Blocks saved to disk.


In [16]:
!unzip 'Deep_Learning_Assignment/dataset/PartBblock_1.npz' -d 'Deep_Learning_Assignment/dataset/PartB/Block1'

Archive:  Deep_Learning_Assignment/dataset/PartBblock_1.npz
 extracting: Deep_Learning_Assignment/dataset/PartB/Block1/x_train.npy  
 extracting: Deep_Learning_Assignment/dataset/PartB/Block1/y_train.npy  
 extracting: Deep_Learning_Assignment/dataset/PartB/Block1/x_test.npy  
 extracting: Deep_Learning_Assignment/dataset/PartB/Block1/y_test.npy  


In [17]:
import numpy as np

x_train = np.load('Deep_Learning_Assignment/dataset/PartB/Block1/x_train.npy')
x_test = np.load('Deep_Learning_Assignment/dataset/PartB/Block1/x_test.npy')

y_train = np.load('Deep_Learning_Assignment/dataset/PartB/Block1/y_train.npy')
y_test = np.load('Deep_Learning_Assignment/dataset/PartB/Block1/y_test.npy')


In [18]:

x_train, x_test = x_train / 255.0, x_test / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 100)
y_test = tf.keras.utils.to_categorical(y_test, 100)

In [19]:
loaded_model = load_model("Deep_Learning_Assignment/B1_model_skip_connection.h5")
y_pred_prob=loaded_model.predict(x_test)
y_pred=np.argmax(y_pred_prob,axis=1)
y_true=np.argmax(y_test,axis=1)
f1=f1_score(y_true,y_pred,average='weighted')
print(f"F1 score:{f1}")
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')

print(f'Precision: {precision}')
print(f'Recall: {recall}')

test_loss, test_accuracy = loaded_model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_accuracy}')
print(f'Loss:{test_loss}')

157/157 [==============================] - 1s 3ms/step
F1 score:0.3230360502692302
Precision: 0.3355385956988175
Recall: 0.3346
157/157 [==============================] - 1s 3ms/step - loss: 2.6587 - accuracy: 0.3346
Test accuracy: 0.3346000015735626
Loss:2.658729314804077


**Autoencoder**

In [20]:
loaded_autoencoder_model = load_model("Deep_Learning_Assignment/Autoencoder_model.h5")
loaded_test_loss= loaded_autoencoder_model.evaluate(x_test, x_test)
print(f'Loss:{test_loss}')

from sklearn.metrics import mean_squared_error


reconstructed_images = loaded_autoencoder_model.predict(x_test)


original_images_flat = x_test.reshape((x_test.shape[0], -1))
reconstructed_images_flat = reconstructed_images.reshape((reconstructed_images.shape[0], -1))

mse = mean_squared_error(original_images_flat, reconstructed_images_flat)

print(f'Mean Squared Error (MSE): {mse}')

157/157 [==============================] - 2s 5ms/step - loss: 0.5526
Loss:2.658729314804077
157/157 [==============================] - 1s 3ms/step
Mean Squared Error (MSE): 0.009726255893291106
